In [1]:
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
import word_preprocessing as wp

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
[nltk_data] Downloading package punkt to /home/kaushik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time']
processor  =  wp.Word_Processor('../grammar/')
feature_set,class_set,lexicon_size = processor.words_to_vectors()


In [3]:
lstm_units = 128
batch_size = 10
lstm_layers = 1
learning_rate = 0.01

def split_sets(data, classes ,split_point = 1):
    split_length  = int(data.shape[0] * split_point)
    training_features = data[:split_length]
    training_ops = classes[:split_length]
    testing_features = data[split_length:]
    testing_ops = classes[split_length:]
    return training_features,training_ops,testing_features,testing_ops

trainX,trainY,_,_ = split_sets(feature_set, class_set)
trainX = np.reshape(trainX, newshape=[trainX.shape[0],trainX.shape[1],1])
print(trainY)


[1 2 2 ..., 2 2 2]


In [4]:
lstm_net =  tflearn.input_data(shape=[None,lexicon_size,1])
print(trainY)
trainY = to_categorical(trainY, nb_classes=3)
print(trainY)
#lstm_net = tflearn.embedding(lstm_net)
lstm_net =  tflearn.lstm(lstm_net,128)
lstm_net = tflearn.fully_connected(lstm_net,n_units=len(ml_classes),activation='softmax')
lstm_net = tflearn.regression(lstm_net, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy')

model = tflearn.DNN(lstm_net, tensorboard_verbose=0)
model.fit(trainX, trainY,show_metric=True,batch_size=10,n_epoch=10)


Training Step: 1709  | total loss: 0.01320 | time: 7.924s
| Adam | epoch: 010 | loss: 0.01320 - acc: 0.9919 -- iter: 1700/1710
Training Step: 1710  | total loss: 0.01198 | time: 7.969s
| Adam | epoch: 010 | loss: 0.01198 - acc: 0.9927 -- iter: 1710/1710
--


In [6]:
test_x = "what are the class timings"
tr_data = processor.vectorize_input(test_x)
tr_data = np.reshape(tr_data, newshape=[1,lexicon_size,1])
predicted_class = model.predict(tr_data)
print(predicted_class)

[[  9.55703435e-05   1.92470870e-05   9.99885201e-01]]
